### NNDL baseline trainings

#### Download the kaggle datasets

In [ ]:
!wget -O released_data.zip "https://www.dropbox.com/s/c2dvapqb613ukhw/Released_Data-20221201T215316Z-001.zip?dl=0"

!unzip released_data.zip

!unzip ./Released_Data/test_shuffle.zip
!unzip ./Released_Data/train_shuffle.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_shuffle/4824.jpg  
  inflating: train_shuffle/3193.jpg  
  inflating: train_shuffle/1784.jpg  
  inflating: train_shuffle/210.jpg   
  inflating: train_shuffle/6000.jpg  
  inflating: train_shuffle/3178.jpg  
  inflating: train_shuffle/4617.jpg  
  inflating: train_shuffle/5509.jpg  
  inflating: train_shuffle/2266.jpg  
  inflating: train_shuffle/2500.jpg  
  inflating: train_shuffle/4171.jpg  
  inflating: train_shuffle/1009.jpg  
  inflating: train_shuffle/589.jpg   
  inflating: train_shuffle/2514.jpg  
  inflating: train_shuffle/4165.jpg  
  inflating: train_shuffle/4603.jpg  
  inflating: train_shuffle/2272.jpg  
  inflating: train_shuffle/6014.jpg  
  inflating: train_shuffle/3144.jpg  
  inflating: train_shuffle/5535.jpg  
  inflating: train_shuffle/1753.jpg  
  inflating: train_shuffle/1035.jpg  
  inflating: train_shuffle/5253.jpg  
  inflating: train_shuffle/3622.jpg  
  inflating: train_shuffle/2528.jpg  

#### Dataset and Dataloader preparation

##### Train and Validation data splits

In [ ]:
import os
import sys
import numpy as np
import pandas as pd

train_imgs = np.array(os.listdir('./train_shuffle'))
train_labels = pd.read_csv('./Released_Data/train_data.csv').to_numpy()

labels_list = [set(),set(),set()]

for i in range(len(train_labels)):
    labels_list[train_labels[i][1]].add(train_labels[i][2])

print(list(labels_list[0]))
print(list(labels_list[1]))
print(list(labels_list[2]))


from sklearn.model_selection import train_test_split

train_data, val_data, train_target, val_target =  train_test_split(train_imgs, train_labels[:, 2], train_size = 0.90, \
                                                                   random_state=5, stratify=train_labels[:, 2], shuffle=True)


full_train_data = []
full_val_data = []

train_count = 0
val_count = 0

for i in range(train_labels.shape[0]):
    name = train_labels[i][0]
    if name in val_data:
        full_val_data.append(train_labels[i, :].tolist())
        val_count += 1
    else:
        full_train_data.append(train_labels[i, :].tolist())
        train_count += 1  

print(train_count)
print(val_count)
print(full_train_data[:5])
print(full_val_data[:5])
print(train_labels[:10])

######### Code to check the distribution of super and sub classes in the training and validation datasets
# train_sub_dict = {}
# train_super_dict = {}
# val_sub_dict = {}
# val_super_dict = {}

# for i in range(train_labels.shape[0]):
#     name = train_labels[i][0]
#     super_class = train_labels[i][1]
#     sub_class = train_labels[i][2]

#     if name in val_data:
#         if super_class in val_super_dict:
#             val_super_dict[super_class] += 1
#         else:
#             val_super_dict[super_class] = 1
        
#         if sub_class in val_sub_dict:
#             val_sub_dict[sub_class] += 1
#         else:
#             val_sub_dict[sub_class] = 1
#     else:
#         if super_class in train_super_dict:
#             train_super_dict[super_class] += 1
#         else:
#             train_super_dict[super_class] = 1
        
#         if sub_class in train_sub_dict:
#             train_sub_dict[sub_class] += 1
#         else:
#             train_sub_dict[sub_class] = 1


# for k in val_super_dict:
#     print(k, val_super_dict[k], train_super_dict[k], train_super_dict[k]/val_super_dict[k])

# for k in val_sub_dict:
#     print(k, val_sub_dict[k], train_sub_dict[k], train_sub_dict[k]/val_sub_dict[k])




[4, 5, 6, 8, 11, 14, 16, 19, 20, 24, 26, 27, 28, 30, 40, 41, 42, 52, 57, 60, 61, 74, 76, 79, 81, 82, 84, 85, 86, 88]
[0, 2, 7, 9, 10, 12, 17, 18, 21, 22, 23, 25, 31, 32, 36, 37, 38, 45, 46, 49, 54, 55, 63, 65, 66, 71, 78, 80, 87]
[1, 3, 13, 15, 29, 33, 34, 35, 39, 43, 44, 47, 48, 50, 51, 53, 56, 58, 59, 62, 64, 67, 68, 69, 70, 72, 73, 75, 77, 83]
5824
648
[['0.jpg', 1, 63], ['1.jpg', 0, 24], ['2.jpg', 0, 74], ['3.jpg', 2, 72], ['5.jpg', 2, 1]]
[['4.jpg', 2, 43], ['7.jpg', 1, 12], ['12.jpg', 1, 18], ['19.jpg', 0, 27], ['24.jpg', 2, 35]]
[['0.jpg' 1 63]
 ['1.jpg' 0 24]
 ['2.jpg' 0 74]
 ['3.jpg' 2 72]
 ['4.jpg' 2 43]
 ['5.jpg' 2 1]
 ['6.jpg' 0 11]
 ['7.jpg' 1 12]
 ['8.jpg' 1 49]
 ['9.jpg' 1 63]]


##### KaggleDataset dataloader

In [ ]:
import os
import sys
import pandas as pd
import cv2
import torch
import torch.nn as nn

import albumentations as A
from albumentations.pytorch import ToTensorV2

class KaggleDataset(nn.Module):
    def __init__(self, mode = 'train'):
        super(KaggleDataset, self).__init__()
        self.mode = mode

        if mode == 'train':
            self.dataset = full_train_data
        else:
            self.dataset = full_val_data
        

    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_name = self.dataset[idx][0]
        img = cv2.imread(os.path.join('train_shuffle', img_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    #convert image from BGR to RGB format

        super_class = torch.tensor(self.dataset[idx][1], dtype = torch.float32)
        sub_class = torch.tensor(self.dataset[idx][2], dtype = torch.float32)

        




    

    def transform_data(self):

        if self.mode == 'train':
            transform_data = A.Compose(
              [
                  #always resize the image to 329x224
                  #A.Resize(height = 329, width= 224, interpolation = cv2.INTER_AREA, p=1),
                  A.HorizontalFlip(p=0.4),  
                  A.ShiftScaleRotate(shift_limit=0.025, scale_limit=0, rotate_limit=15, p=0.5),
                  #A.RandomCrop(height = 224, width = 224, p=1),
                  #randomly change brightness, contrast, and saturation of the image 50% of the time
                  A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue = 0, p=0.5), 
                  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1), 
                  ToTensorV2(p=1),
              ])
        else:     #augmentations during validation and testing
          transform_data = A.Compose(
          [
              #always resize the image to 329x224
              #A.Resize(height = 329, width = 224, p=1),   
              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1),
              ToTensorV2(p=1),
          ])
    
        return transform_data







6472
9127
(6472, 3)
